In [6]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Birgit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
articles = pd.read_json("data/articles.json.xz")
jokes = pd.read_json("data/jokes.json.xz")

In [3]:
ajalehed = ["Washington Post", "The New York Times", "Reuters"]

In [8]:
re.sub("The New York Times", "", 'The New York Times reported on Thursday that The New York Times Trump backed down from his order after the White House counsel threatened to resign rather than follow his directive, citing four people told of the matter. “Fake news, folks, fake news,” Trump told reporters in Davos, when asked about the report. Reporting by Steve Holland')

' reported on Thursday that  Trump backed down from his order after the White House counsel threatened to resign rather than follow his directive, citing four people told of the matter. “Fake news, folks, fake news,” Trump told reporters in Davos, when asked about the report. Reporting by Steve Holland'

In [22]:
new_jokes = {}

for i, row in jokes.iterrows():
    joke = row['text']
    if "?" not in joke:
        new_jokes[i] = {'text': joke}

In [25]:
non_question_jokes = pd.DataFrame(new_jokes).T

In [26]:
non_question_jokes.shape

(360085, 1)

In [29]:
datas = {}
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stop_words.extend(["...", "'s", "wo", "n't", "'m", "ca", "'ll", "'re", "'ve", "'d", "ha", "´´", "´", "´´´", ",", "!", "?", "'", ":", ";", '"', "\\", "``"])

all_lem_data = []
for name, data, n in zip(['articles', 'jokes'], [articles, non_question_jokes], [100000, 100000]):
    print(name, end="\n")
    lemmatized_data = []
    for i, row in data.iterrows():
        if i == n:
            break
        if i % 10000 == 0:
            print(i)
        text = row['text']
        for ajaleht in ajalehed:
            text = re.sub(ajaleht, "", text)
        
        words = word_tokenize(text)
        lemmas = []

        for word in words:
            lemma = lemmatizer.lemmatize(word)
            if len(lemma) > 1 and not lemma in stop_words:
                lemmas.append(lemma.lower())
                
        lemmatized_row = {'id': f"{name}_{i}",'text': " ".join(lemmas), 'joke': 1 if name == 'jokes' else 0}
        lemmatized_data.append(lemmatized_row)
        all_lem_data.append(lemmatized_row)
    
    datas[name] = pd.DataFrame(lemmatized_data)

articles
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
jokes
0
10000
40000
50000
90000
110000
130000
140000
170000
190000
230000
240000
260000
270000
290000
300000
340000
370000
390000
410000
420000
430000
460000
480000
510000
520000
540000
550000
560000
570000
610000
630000
690000
720000
760000


In [48]:
lemmatized_jokes = list(datas['jokes'][:100000].text)

In [49]:
lemmas_jokes = []

for joke in lemmatized_jokes:
    lemmas_jokes.extend(joke.split(" "))

In [51]:
from collections import Counter

In [64]:
lemmatized_articles = list(datas['articles'].text)

In [65]:
lemmas_articles = []

for article in lemmatized_articles:
    lemmas_articles.extend(article.split(" "))

In [66]:
len(lemmas_articles)

6636431

In [75]:
stopwords = ["wa", "''", "the", "he", "my", "it"]

In [83]:
lemmas_articles = [lemma for lemma in lemmas_articles if lemma not in stopwords and lemma.isalpha()]
lemmas_jokes = [lemma for lemma in lemmas_jokes if lemma not in stopwords and lemma.isalpha()]

In [84]:
most_frequent_jokes = Counter(lemmas_jokes)

In [85]:
most_frequent_jokes.most_common()[:20]

[('say', 15877),
 ('one', 14062),
 ('man', 13241),
 ('said', 12493),
 ('like', 11611),
 ('get', 11392),
 ('go', 9380),
 ('joke', 8716),
 ('day', 8327),
 ('got', 8240),
 ('time', 7636),
 ('people', 7221),
 ('wife', 7215),
 ('guy', 6571),
 ('they', 6505),
 ('know', 6404),
 ('you', 6282),
 ('told', 6101),
 ('would', 5982),
 ('but', 5937)]

In [86]:
most_frequent_articles = Counter(lemmas_articles)

In [87]:
most_frequent_articles.most_common()[:20]

[('said', 106761),
 ('year', 33481),
 ('would', 29712),
 ('percent', 26549),
 ('trump', 25108),
 ('company', 23870),
 ('last', 21266),
 ('reporting', 20534),
 ('new', 19794),
 ('also', 19651),
 ('editing', 18326),
 ('president', 17993),
 ('two', 17623),
 ('million', 17593),
 ('told', 17390),
 ('government', 17276),
 ('billion', 17045),
 ('bank', 16516),
 ('one', 15653),
 ('market', 15596)]

In [89]:
for word in lemmas_articles:
    if not word.isalpha():
        print(word)

### Final preprocessing of jokes and articles

In [90]:
datas['jokes'].head()

,id,text,joke
0,jokes_0,seafood diet see food fish eat,1
1,jokes_1,the shoe store an al bundy one-liner fat woman...,1
2,jokes_3,confused man see psychiatrist man go see psych...,1
3,jokes_8,feel like fluffy white cloud azure blue sky high,1
4,jokes_10,stalin said dark humor like food not everyone get,1


In [139]:
data_for_df = []

jokes = list(datas['jokes'][:100000].text)
ids = list(datas['jokes'][:100000].id)

for i, joke in enumerate(jokes):
    joke_words = joke.split(" ")
    joke_words = [word for word in joke_words if word not in stopwords and word.isalpha()]
    joke_text = " ".join(joke_words)
    data_for_df.append({'id': ids[i], 'text': joke_text, 'joke': 1})
    
articles = list(datas['articles'].text)
article_ids = list(datas['articles'].id)

for i, article in enumerate(articles):
    article_words = article.split(" ")
    article_words = [word for word in article_words if word not in stopwords and word.isalpha()]
    article_text = " ".join(article_words)
    atricle_text = re.sub("reporting .*", "", article_text)
    data_for_df.append({'id': article_ids[i], 'text': article_text, 'joke': 0})

In [140]:
df = pd.DataFrame(data_for_df)

In [141]:
df

,id,text,joke
0,jokes_0,seafood diet see food fish eat,1
1,jokes_1,shoe store an al bundy fat woman came shoe sto...,1
2,jokes_3,confused man see psychiatrist man go see psych...,1
3,jokes_8,feel like fluffy white cloud azure blue sky high,1
4,jokes_10,stalin said dark humor like food not everyone get,1
...,...,...,...
199995,articles_99995,group lawmaker heart logjam said backed aoun e...,0
199996,articles_99996,sale would give pwc freer hand pursue growing ...,0
199997,articles_99997,pressure building company almost two year defe...,0
199998,articles_99998,strong earnings season would help justify pric...,0


### Model

In [115]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [101]:
from sklearn.model_selection import train_test_split

In [157]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['joke'], test_size=0.05, stratify=df['joke'])

In [158]:
y_test.value_counts()

1    5000
0    5000
Name: joke, dtype: int64

In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [147]:
num_words = 15000
#maxlen = 500

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train[:10000])

tokenized_X_train = tokenizer.texts_to_sequences(X_train[:10000])
tokenized_X_test = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(tokenized_X_train)
X_test_pad = pad_sequences(tokenized_X_test, maxlen=X_train_pad.shape[1])

In [148]:
y_train_vec = to_categorical(y_train[:10000])
y_test_vec = to_categorical(y_test)

In [149]:
X_train_pad.shape

(10000, 594)

In [150]:
X_test_pad.shape

(2000, 594)

In [151]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=num_words, input_length=X_train_pad.shape[1], output_dim=64))
model.add(layers.LSTM(35))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 594, 64)           960000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 35)                14000     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 72        
Total params: 974,072
Trainable params: 974,072
Non-trainable params: 0
_________________________________________________________________


In [153]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [154]:
model.fit(X_train_pad, y_train_vec, validation_data=(X_test_pad, y_test_vec), epochs=3, verbose=1, batch_size=1024)

Train on 10000 samples, validate on 2000 samples
Epoch 1/3
10000/10000 [==============================] - 422s 42ms/sample - loss: 0.6800 - accuracy: 0.7823 - val_loss: 0.6548 - val_accuracy: 0.9535
Epoch 2/3
10000/10000 [==============================] - 442s 44ms/sample - loss: 0.6074 - accuracy: 0.9186 - val_loss: 0.5184 - val_accuracy: 0.8770
Epoch 3/3
10000/10000 [==============================] - 541s 54ms/sample - loss: 0.4343 - accuracy: 0.8632 - val_loss: 0.3344 - val_accuracy: 0.8760


#### Second model

In [159]:
num_words = 15000
#maxlen = 500

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train)

tokenized_X_train = tokenizer.texts_to_sequences(X_train)
tokenized_X_test = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(tokenized_X_train)
X_test_pad = pad_sequences(tokenized_X_test, maxlen=X_train_pad.shape[1])

In [160]:
y_train_vec = to_categorical(y_train)
y_test_vec = to_categorical(y_test)

In [161]:
model2 = keras.Sequential()
model2.add(layers.Embedding(input_dim=num_words, input_length=X_train_pad.shape[1], output_dim=64))
model2.add(layers.LSTM(35))
model2.add(layers.Dense(2, activation='softmax'))
model2.summary()
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 3835, 64)          960000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 35)                14000     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 72        
Total params: 974,072
Trainable params: 974,072
Non-trainable params: 0
_________________________________________________________________


In [163]:
model2.fit(X_train_pad, y_train_vec, validation_data=(X_test_pad, y_test_vec), epochs=3, verbose=1, batch_size=64)

Train on 190000 samples, validate on 10000 samples
Epoch 1/3
 13440/190000 [=>............................] - ETA: 5:53:11 - loss: 0.1642 - accuracy: 0.9578

KeyboardInterrupt: 

### More data exploration

In [134]:
reporting = 0

for i, t in enumerate(datas['articles'].text):
    if "reporting " in t:
        #print(t)
        #print("-----------------------------------")
        reporting += 1
    #if i == 100:
    #    break

In [135]:
reporting

20248

In [137]:
for i, t in enumerate(datas['articles'].text):
    if "reporting " in t:
        print(t)
        print("\n")
        print(re.sub("reporting .*", "", t))
        print("-----------------------------------")
    if i == 100:
        break

reported thursday trump backed order white house counsel threatened resign rather follow directive citing four people told matter fake news folk fake news trump told reporter davos asked report reporting steve holland


reported thursday trump backed order white house counsel threatened resign rather follow directive citing four people told matter fake news folk fake news trump told reporter davos asked report 
-----------------------------------
so-called tiered deposit rate would mean bank exempted part paying ecb 0.40 percent annual charge excess reserve boosting profit struggle unexpected growth slowdown we would consider whether tiering system needed coeure said interview financial times today prevailing view governing council also agree deserves reflection. reporting michelle martin editing tassilo hummel


so-called tiered deposit rate would mean bank exempted part paying ecb 0.40 percent annual charge excess reserve boosting profit struggle unexpected growth slowdown we would c